# Useful functions

In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt

from matplotlib.patches import Polygon as pltpolygon
from matplotlib.collections import PatchCollection

import import_ipynb
import node_linker
import machines

importing Jupyter notebook from node_linker.ipynb
importing Jupyter notebook from machines.ipynb


## Point in a Segment Function

`def between(a, b, c)`

Check inclusion of a point in a segment

### Parameters:
* **a : list of float**
    * First vertex of the segment.
* **b : list of float**
    * Second vertex of the segment.
* **c : list of float**
    * Point to check.
    
### Return:
*  **True/False**
    * True if the point is inside the line.

In [ ]:
def between(a, b, c):
    crossproduct = (c[1] - a[1])*(b[0] - a[0]) - (c[0] - a[0])*(b[1] - a[1])
    # compare versus epsilon for floating point values, or != 0 if using
    # integers
    if abs(crossproduct) > 0.01:
        return False
    dotproduct = (c[0] - a[0])*(b[0] - a[0]) + (c[1] - a[1])*(b[1] - a[1])
    if dotproduct < 0:
        return False
    squaredlengthba = (b[0] - a[0])*(b[0] - a[0]) + (b[1] - a[1])*(b[1] - a[1])
    if dotproduct > squaredlengthba:
        return False
    return True

## Collision Free Configuration Function

`def collision(neighbors, radius, robots)`

Test if the initial position of robots is collision free configuration.

### Parameters:
* **neighbors : Voronoi.ridge_points**
    * List of pairs of neighbors
* **radius&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: float**
    * Safety radius to define the BVC from the Voronoi Diagram
* **robots&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: list**
    * List of instances of Robots
    
### Return:
* **crash&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: bool**
    * True if collision exists
* **message : string**
    * Communicating what configuration is creating conflicts: initial or final 

In [73]:
def collision(neighbors, radius, robots):
    crash = False
    message = ""
    i = 0
    while i < len(neighbors) and not crash:
        if (np.linalg.norm(robots[neighbors[i][1]].pos_i
                           - robots[neighbors[i][0]].pos_i) < 2*radius):
            crash = True
            message = "ERROR!: Change the initial configuration"
        elif (np.linalg.norm(robots[neighbors[i][1]].pos_f
                             - robots[neighbors[i][0]].pos_f) < 2*radius):
            crash = True
            message = "ERROR!: Change the final configuration" 
        i += 1
    return {"crash" : crash, "message" : message}

## Palette Function

`def colors(number):`

Assign a color to each agent

### Parameters:
* **number : int**
    * Number of agents
* **robots&nbsp;&nbsp;&nbsp;: list**
    * List of instances of Robots

In [1]:
def colors(number, robots):
    palette = ['red','tomato','orangered','orange','goldenrod','gold',
               'yellowgreen','chartreuse','lime','green','darkgreen','teal',
               'turquoise','aqua','cyan','blue','navy','indigo','purple',
               'fuchsia','magenta','crimson','maroon','brown','sienna']
    for i in range(number):
        robots[i].color = palette[int(i*(25/number))]

## Intersection Function

`def intersect(origin, radius, vertice1, vertice2):`

Generate the new BVC vertices from the intersection of the Voronoi edges  
moved according the safety radius in different sectors of the cell

### Parameters:
* **origin&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: ndarray of float**
    * Coordinates from one of the sector intermediate Voronoi vertex.
* **radius&nbsp;&nbsp;&nbsp;&nbsp;: float**
    * radius to define the BVC from the Voronoi Diagram
* **vertice1&nbsp;: ndarray of float**
    * Coordinates from one of the sector extreme Voronoi vertex.
* **vertice2 : ndarray of float**
    * Coordinates from one of the sector extreme Voronoi vertex.

### Return:
* **new_vertice1 : ndarray of float**
    * Coordinates from one of the sector extreme BVC vertex.
* **intersection&nbsp;&nbsp;&nbsp;: ndarray of float**
    * Coordinates from one of the sector intermediate BVC vertex.
* **new_vertice2 : ndarray of float**
    * Coordinates from one of the sector extreme BVC vertex.

In [4]:
def intersect(origin, radius, vertice1, vertice2):
    # Change coordinates
    vertice1 = vertice1 - origin
    vertice2 = vertice2 - origin
    # Unit vectors
    vertice1_unit = vertice1 / np.linalg.norm(vertice1)
    vertice2_unit = vertice2 / np.linalg.norm(vertice2)
    # Get the angle between edges
    p_dot = np.dot(vertice1_unit, vertice2_unit)
    angle = np.arccos(p_dot)
    mid_angle = angle/2
    # Compute and show the intersection of the BVC
    mid_vertex = (vertice1_unit + vertice2_unit) / 2
    mid_vertex_unit = mid_vertex / np.linalg.norm(mid_vertex)
    separation = radius / np.sin(mid_angle)
    intersection = mid_vertex_unit * separation
    # Revert coordinates
    new_vertice1 = vertice1 + intersection + origin
    new_vertice2 = vertice2 + intersection + origin
    intersection = intersection + origin
    return [new_vertice1, intersection, new_vertice2]

## Vertices Sense Function

`def get_sense(prev_pos, vertices)`

Get the sense of a vertices set

### Parameters:
* **prev_pos : ndarray of floats**
    * Coordinates of the previous positions of robots.
* **vetices&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: list of ndarrays of float**
    * Current positions of the robots.
    
### Return:
* **sense : string**
    * "clockwise" or counterclockwise" 

In [ ]:
def get_sense(prev_pos, vertices):
    #print("Vertex 0 type:", type(vertices[0]))
    #print("Vertex 1 type:", type(vertices[1]))
    #print("Vertex 2 type:", type(vertices[2]))
    #print("Robots path type:", type(prev_pos),"\n")
    # Check the type of cell
    if len(vertices) > 2:
        midpoint = vertices[1]
        extreme1 = vertices[0] - midpoint
        extreme2 = vertices[2] - midpoint
    else:
        midpoint = prev_pos
        extreme1 = vertices[0] - midpoint
        extreme2 = vertices[1] - midpoint
    # Analyze the crossproduct
    crossproduct = extreme1[0]*extreme2[1] - extreme2[0]*extreme1[1]
    if crossproduct > 0:
        sense = 'clockwise'
    elif crossproduct < 0:
        sense = 'counterclockwise'
    return sense

## Assign Neighbors

`def neighborhood(robots, ridge_points)`

Inserting information in the machines.Robot.neighbors id and pos

### Parameters:
* **robots&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
: list**
    * List of instances of Robots
* **ridge_points : Voronoi.ridge_points**
    * List of pairs of neighbors

In [1]:
def neighborhood(robots, ridge_points):
    for agent in robots:
        agent.neighbors_id = []
        agent.neighbors_pos = []
        for in_front in ridge_points:
            if agent.id == in_front[0]:
                agent.neighbors_id.append(robots[in_front[1]].id)
                agent.neighbors_pos.append(robots[in_front[1]].pos_i)
            elif agent.id == in_front[1]:
                agent.neighbors_id.append(robots[in_front[0]].id)
                agent.neighbors_pos.append(robots[in_front[0]].pos_i)

## Closer Edge Points Function

`def perpendicular_dist(target_point, vertices)`

Get the closest point of an edge to the target point

### Parameters:
* **target_point : ndarray of float**
    * The target point of a specific robot.
* **vertices&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;:
 ndarray of float**
    * Finite BVC vertices.
    
### Return:
* **edge_points : ndarray of float**
    * List of closest points on the BVC edges to the target

In [ ]:
def perpendicular_dist(target_point, vertices):
    #Initialize the list to return
    edge_points = []
    # Analize each edge
    for i in range(len(vertices[:-1])):
        # Check if the edge is vertical
        if vertices[i][0] == vertices[i+1][0]:
            [x, y] = [vertices[i][0], target_point[1]]
        else:
            # Get the perpendicular point on the egde sense to the target
            slope = (vertices[i+1][1] - vertices[i][1])/(vertices[i+1][0]
                                                         - vertices[i][0])
            x = (slope * (vertices[i][1] - target_point[1]
                          - slope * vertices[i][0])
                 - target_point[0]) / (-1 - pow(slope,2))
            y = (x - target_point[0])/(-slope) + target_point[1]
            # Check if the point is inside of edge to include in the returning
            # list
        if between(vertices[i], vertices[i+1], [x,y]):
            edge_points.append([x,y])
    return edge_points

## Plotter

`def plot(opt, robots, vor_prox)`

Function to plot the elements of the simulation: Voronoi diagram, BVC, robots,
paths, positions and closer points

### Parameters:
* **opt&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
: dictionary**
    * Dictionary with general options
* **robots&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: list**
    * List of instances of Robots
* **vor_prox : dictionary**
    * Approximated values for Voronoi generation

In [ ]:
def plot(opt, robots, vor_prox):
    fig, ax = plt.subplots()
    for agent in robots:
        patches = []
        if opt["zoom"]:
            plt.xlim(-1.05,1.05); plt.ylim(-0.75,0.75)
            #plt.xlim(-0.53,0.53); plt.ylim(-0.38,0.38)
        else:
            constant = math.sqrt(2*len(robots)) + (1/(0.4*len(robots)))
            plt.xlim((constant)*-1.4, (constant)* 1.4)
            plt.ylim(-(constant), constant)
        # Display edges
        for i in range(len(agent.bvc)-1):
            plt.plot([agent.bvc[i][0], agent.bvc[i+1][0]],
                     [agent.bvc[i][1], agent.bvc[i+1][1]],
                     color = agent.color, linestyle = '-')
        # Plot robots, target, line to target,robot path and new direction
        plt.plot(agent.pos_i[0], agent.pos_i[1], color=agent.color, marker='o')
        plt.plot(agent.pos_f[0], agent.pos_f[1], color=agent.color, marker='o')
        plt.plot([agent.pos_i[0], agent.pos_f[0]],
                 [agent.pos_i[1], agent.pos_f[1]],
                 color = 'gray', linestyle = '--')
        # BVC Cell inside
        inst_polygon = pltpolygon(np.array(agent.bvc), True)
        patches.append(inst_polygon)
        p = PatchCollection(patches, alpha=0.1)
        p.set_color(agent.color)
        ax.add_collection(p)
        # Plot path
        for i in range(len(agent.path)-1):
            inst_start = agent.path[i]
            inst_end = agent.path[i+1]
            plt.plot([inst_start[0], inst_end[0]], [inst_start[1],inst_end[1]],
                     color = agent.color, linestyle = '-')
        plt.plot([agent.pos_i[0], agent.closer_point[0]],
                 [agent.pos_i[1], agent.closer_point[1]], 'k-')
        # Display the Voronoi Diagram
        for arista in vor_prox["edge"]:
            x_values = [vor_prox["vrtx"][arista[0],0],
                        vor_prox["vrtx"][arista[1],0]]
            y_values = [vor_prox["vrtx"][arista[0],1],
                        vor_prox["vrtx"][arista[1],1]]
            plt.plot(x_values, y_values, color = 'gray', linestyle = '-')

## Position Generation

`pos_gen(num, robots, seed)`

Generate a pseudorandom configurations for initial and final positions

### Parameters:
* **robots : list**
    * List of instances of Robots
* **num&nbsp;&nbsp;&nbsp;&nbsp;: int**
    * Number of robots
* **seed&nbsp;&nbsp;&nbsp;&nbsp;: int**
    * Seed for pseudorandom number generation

In [9]:
def pos_gen(num, robots, seed):
    np.random.seed(seed)
    # Circle radius
    radius = math.sqrt(2*num)
    offset = (1/(0.4*num) * np.random.random_sample((num*4,)) - 1/(0.4*num))
    for i in range(0, num):
        convert = (2*math.pi*i)/num
        robots[i].pos_i = np.array([round(radius*math.cos(convert)
                                          + offset[4*i],2),
                                    round(radius*math.sin(convert)
                                          + offset[4*i+1],2)])
        robots[i].pos_f = np.array([round(radius*math.cos(convert+math.pi)
                                          + offset[4*i+2],2),
                                    round(radius*math.sin(convert+math.pi)
                                          + offset[4*i+3],2)])
        robots[i].dist = np.linalg.norm(robots[i].pos_f - robots[i].pos_i)

## RHC Plotter

`def plot_rhc(u, p, opt, step)`

Function to plot the elements of the solver.

### Parameters:
* **u&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: CVXPY Variable**
    * Restrictions of movement.
* **p&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: CVXPY Variable**
    * Positions calculated.
* **opt&nbsp;&nbsp;&nbsp;: dictionary**
    * Dictionary with general options.
* **step : int**
    * Global step of the execution.

In [ ]:
def plot_rhc(u, p, opt, step):
   # Plot results.
    %matplotlib inline
    %config InlineBackend.figure_format = 'svg'
    f = plt.figure()
    # Plot (u_t)_1.
    ax = f.add_subplot(411)
    plt.plot(u[0,:].value)
    plt.ylabel(r"$(u_t)_1$", fontsize=16, color='black')
    plt.yticks(np.linspace(-1.0, 1.0, 3), color='black')
    plt.xticks([])
    # Plot (u_t)_2.
    plt.subplot(4,1,2)
    plt.plot(u[1,:].value)
    plt.ylabel(r"$(u_t)_2$", fontsize=16, color='black')
    plt.yticks(np.linspace(-1, 1, 3), color='black')
    plt.xticks([])
    # Plot (x_t)_1.
    plt.subplot(4,1,3)
    p1 = p[0,:].value
    plt.plot(p1)
    plt.ylabel(r"$(p_t)_1$", fontsize=16, color='black')
    plt.yticks([-6, 0, 6], color='black')
    plt.ylim([-6, 6])
    plt.xticks([])
    # Plot (x_t)_2.
    plt.subplot(4,1,4)
    p2 = p[1,:].value
    plt.plot(range(21), p2)
    plt.yticks([-6, 0, 6], color='black')
    plt.ylim([-6, 6])
    plt.ylabel(r"$(p_t)_2$", fontsize=16, color='black')
    plt.xlabel(r"$t$", fontsize=16, color='black')
    plt.xticks(color='black')
    plt.tight_layout()
    if opt["save_image"]:
        plt.savefig(opt["path_image"]+'solver_'+step+'.svg')
    elif opt["plot"]: 
        plt.close()

### Pruner Function

`def pruner(agent, verbose)`

Remove intersections of the BVC

#### Parameters:
* **agent&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: class machines.Robot**
    * Current agent.
* **verbose : boolean**
    * Flag to print important values

In [ ]:
def pruner(agent, verbose):
    pruned_vertices = []
    cell = node_linker.DoublyLinkedList()
    for vertex in agent.bvc:
        cell.insert_at_end(vertex)
    runner = cell.start_node
    if verbose:
        print('Vertices in doubly linked list:')
        cell.traverse_list()
    while not(runner is cell.end_node.pref.pref
          or runner is cell.end_node.pref):
        vanguard = runner.nref.nref
        while vanguard is not cell.end_node:
            vertex_0 = runner.item
            vertex_1 = runner.nref.item
            vertex_2 = vanguard.item
            vertex_3 = vanguard.nref.item
            if verbose:
                print("\nChecking: ",vertex_0,"—",vertex_1,"\nand ",
                      vertex_2,"—",vertex_3)
            critical_point = sharpen(vertex_0, vertex_3, vertex_1, vertex_2)
            intersection = between(vertex_0,vertex_1,critical_point)
            if verbose:
                print('Intersection?:',intersection)
            if intersection:
                inst_node = runner.nref
                while inst_node is not vanguard:
                    inst_node = inst_node.nref
                    cell.delete_node(inst_node.pref)
                vanguard.item = critical_point
                if verbose:
                    print('The new value is: ',vanguard.item)
                    print('Connected with: ',vanguard.nref.item)
            vanguard = vanguard.nref
        runner = runner.nref
    runner = cell.start_node
    while runner is not cell.end_node.nref:
        pruned_vertices.append(runner.item)
        runner = runner.nref
    if verbose:
        print("\nCell size: ", cell.lenght_list)
        cell.traverse_list()
    agent.bvc = pruned_vertices

### Save Image

`def save_image(num, opt, step)`

Save image in the computer

#### Parameters:
* **num : int**
    * Number of robots
* **opt&nbsp;&nbsp;&nbsp;: dictionary**
    * Dictionary with general options.
* **step : int**
    * Global step of the execution.

In [ ]:
def save_image(num, opt, step):
    if opt["save_image"]:
        folder = 'Simple'
        if opt["optimize"]:
            folder = 'Opt'
        plt.savefig(opt["path_image"]+folder+'/'+str(num)+'/fig'+str(step)
                    +'.svg')
        print('Image '+str(step)+' saved successfully!')
        plt.close()
    elif opt["plot"]:
        plt.show()

## Sharpen Function

`def sharpen(aux1, aux2, vertex1, vertex2)`

Calculate a new vertex when edges smaller than the safety radius appear 

### Parameters:
* **aux1&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Auxiliary coordinate for vertex1 to generate the peak
* **aux2&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Auxiliary coordinate for vertex2 to generate the peak
* **vertex1&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * One of the edge vertices
* **vertex2&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * The other edge vertices
    
### Return:
* **peak&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * New coordinate that replace the edge

In [ ]:
def sharpen(aux1, aux2, vertex1, vertex2):
    # Special cases: vertical edge
    if aux1[0] == vertex1[0]:
        slope2 = (vertex2[1] - aux2[1])/(vertex2[0] - aux2[0])
        x = aux1[0]
        y = slope2*(x - aux2[0]) + aux2[1]
    elif aux2[0] == vertex2[0]:
        slope1 = (vertex1[1] - aux1[1])/(vertex1[0] - aux1[0])
        x = aux2[0]
        y = slope1*(x - aux1[0]) + aux1[1]
    # Another cases
    else:
        slope1 = (vertex1[1] - aux1[1])/(vertex1[0] - aux1[0])
        slope2 = (vertex2[1] - aux2[1])/(vertex2[0] - aux2[0])
        x = ((vertex2[1] - aux1[1] + slope1*aux1[0] - slope2*vertex2[0]) /
             (slope1 - slope2))
        y = slope1*(x - aux1[0]) + aux1[1]
    peak = np.array([x,y])
    return peak

## Movement Function

`def update_pos(mov, robots)`

Move the agent according the case.

### Parameters:
* **robots : list**
    * List of instances of Robots
* **mov : float**
    * Movement magnitude

In [ ]:
def update_pos(mov, robots):
    for agent in robots:
        int_paths = agent.closer_point - agent.pos_i
        if np.all(int_paths == 0):
            # Reached position
            if not agent.deadlock:
                agent.pos_i = agent.pos_f
        else:
            candidate = (agent.pos_i + ((int_paths / np.linalg.norm(int_paths))
                                        * mov))
            # Reaching the final position
            if between(agent.pos_i, candidate, agent.pos_f):
                fin_dist = agent.pos_i - agent.pos_f
                agent.dist_travel += np.linalg.norm(fin_dist)
                agent.pos_i = agent.pos_f
            # Internal Path less than alpha
            elif (np.linalg.norm(int_paths) < mov):
                agent.pos_i = agent.closer_point
                agent.dist_travel += np.linalg.norm(int_paths)
            # Normal movement
            else:
                agent.pos_i = candidate
                agent.dist_travel += mov